In [1]:
import os
import json
from pymongo.mongo_client import MongoClient
from pymongo import GEOSPHERE
from shapely.geometry import MultiPolygon, Polygon
from shapely import wkt
from PIL import Image, ImageDraw

In [2]:
def process_and_insert_image_data(json_file_path, pre, post):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
        
    base_name = os.path.basename(json_file_path)
    image_name = os.path.splitext(base_name)[0]

    latitudes = []
    longitudes = []

    # Extract latitude and longitude values from the data
    for feature in data['features']['lng_lat']:
        # Extract the points from the POLYGON WKT string
        polygon_text = feature['wkt'].strip('POLYGON ((').strip('))')
        polygon_points = polygon_text.split(', ')

        for point_text in polygon_points:
            longitude, latitude = map(float, point_text.split(' '))
            latitudes.append(latitude)
            longitudes.append(longitude)

    # Calculate the minimum and maximum latitude and longitude
    min_lat = min(latitudes)
    max_lat = max(latitudes)
    min_lon = min(longitudes)
    max_lon = max(longitudes)

    # Construct the bounding box in GeoJSON format
    bounding_box = {
        "type": "Polygon",
        "coordinates": [[
            [min_lon, max_lat],  # Northwest corner
            [max_lon, max_lat],  # Northeast corner
            [max_lon, min_lat],  # Southeast corner
            [min_lon, min_lat],  # Southwest corner
            [min_lon, max_lat]   # Closing the polygon
        ]]
    }

    client = MongoClient(uri)

    db = client['ncsa']
    images_collection = db['ncsa']

    # Create a 2dsphere index on the 'bounding_box' field for geospatial queries
    images_collection.create_index([("bounding_box", GEOSPHERE)])

    image_data = {
        "name": image_name,
        "bounding_box": bounding_box,
        "pre": "https://storage.cloud.google.com/ncsa-hackthon/image/joplin-tornado_00000001_pre_disaster.png",
        "post": "https://storage.cloud.google.com/ncsa-hackthon/image/joplin-tornado_00000001_post_disaster.png",
    }

    images_collection.insert_one(image_data)


In [ ]:
json_path = 'data/joplin-tornado_00000001_pre_disaster.json'
process_and_insert_image_data(json_path)

In [2]:
def process_and_insert_image_data(json_file_path, uri):
    # Load JSON data
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    
    # Extract base name and image name
    base_name = os.path.basename(json_file_path)
    image_name = os.path.splitext(base_name)[0]

    # Convert WKT polygons to Shapely polygons and create a MultiPolygon
    polygons = [Polygon(wkt.loads(feature['wkt'])) for feature in data['features']['lng_lat']]
    multi_polygon = MultiPolygon(polygons)

    # Get the bounding box of the MultiPolygon
    min_lng, min_lat, max_lng, max_lat = multi_polygon.bounds

    # Connect to MongoDB
    client = MongoClient(uri)
    db = client['ncsa']
    images_collection = db['ncsa']
    
    # Create a 2dsphere index on the 'bounding_box' field for geospatial queries
    images_collection.create_index([("bounding_box", "2dsphere")])

    # Construct the bounding box in GeoJSON format
    bounding_box = {
        "type": "Polygon",
        "coordinates": [[
            [min_lng, max_lat],  # Northwest corner
            [max_lng, max_lat],  # Northeast corner
            [max_lng, min_lat],  # Southeast corner
            [min_lng, min_lat],  # Southwest corner
            [min_lng, max_lat]   # Closing the polygon
        ]]
    }

    image_data = {
        "name": image_name,
        "bounding_box": bounding_box,
        "pre": "https://storage.cloud.google.com/ncsa-hackthon/image/joplin-tornado_00000001_pre_disaster.png",
        "post": "https://storage.cloud.google.com/ncsa-hackthon/image/joplin-tornado_00000001_post_disaster.png",
    }
    images_collection.insert_one(image_data)

In [6]:
json_path = 'label/joplin-tornado_00000001_pre_disaster.json'
uri = os.getenv('MONGO')
process_and_insert_image_data(json_path, uri)